<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2025_jun/blob/main/04pln/clase_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 📘 Visualizar textos en TensorFlow Projector: Paso a Paso en Google Colab

Este tutorial explica cómo transformar textos (por ejemplo, artículos constitucionales) en vectores para ser visualizados en [https://projector.tensorflow.org](https://projector.tensorflow.org).

---

## 🔹 Paso 1: Cargar tus textos

Simulamos una lista de artículos constitucionales:

```python
textos = [
    "Artículo 1: Chile es una república democrática, fundada en la soberanía popular.",
    "Artículo 2: La soberanía reside en el pueblo de Chile.",
    "Artículo 3: El Estado se organiza en tres poderes independientes.",
    "Artículo 4: La Constitución garantiza la igualdad ante la ley.",
    "Artículo 5: El medio ambiente debe ser protegido por el Estado.",
]
```

---

## 🔹 Paso 2: Instalar librerías necesarias

Usamos `sentence-transformers` para generar buenos embeddings multilingües:

```python
!pip install -q sentence-transformers
```

---

## 🔹 Paso 3: Convertir los textos a vectores (embeddings)

```python
from sentence_transformers import SentenceTransformer
import pandas as pd

# Cargar modelo multilingüe
modelo = SentenceTransformer('distiluse-base-multilingual-cased')

# Crear los vectores
vectores = modelo.encode(textos)

# Guardamos los textos como etiquetas para visualización
etiquetas = textos
```

---

## 🔹 Paso 4: Guardar archivos `.tsv`

```python
# Guardar los vectores en vectors.tsv
pd.DataFrame(vectores).to_csv("vectors.tsv", sep="\t", header=False, index=False)

# Guardar los textos originales como metadatos
pd.DataFrame(etiquetas).to_csv("metadata.tsv", sep="\t", header=False, index=False)
```

---

## 🔹 Paso 5: Descargar los archivos a tu computador

Puedes usar el explorador de archivos lateral en Colab o ejecutar:

```python
from google.colab import files
files.download("vectors.tsv")
files.download("metadata.tsv")
```

---

## 🔹 Paso 6: Cargar en TensorFlow Projector

1. Ir a: [https://projector.tensorflow.org/](https://projector.tensorflow.org/)
2. Hacer clic en “**Load**”
3. Subir:
   - `vectors.tsv` como **Tensor**
   - `metadata.tsv` como **Metadata**
4. Explorar con **PCA, t-SNE o UMAP**, y buscar similitudes entre textos.

---

## ✅ Opcional: Leer textos desde archivo `.txt` o `.csv`

Si tienes los artículos en un archivo `constitucion.txt` (una línea por artículo), haz:

```python
# Subir archivo
from google.colab import files
uploaded = files.upload()

# Leer textos desde el archivo
with open("constitucion.txt", encoding="utf-8") as f:
    textos = [line.strip() for line in f if line.strip()]
```

---

## 🎉 ¡Listo!
Ahora puedes explorar tus textos en un espacio semántico con TensorFlow Projector.
